# 6 AutoGen - Human interaction

AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. AutoGen agents are customizable, conversable, and seamlessly allow human participation. They can operate in various modes that employ combinations of LLMs, human inputs, and tools.

31/05/2024

<img src="https://raw.githubusercontent.com/retkowsky/images/master/autogenlogo.jpg">

> Documentation: https://microsoft.github.io/autogen/

In [1]:
import autogen
import json
import openai
import os
import sys

from autogen import config_list_from_json

In [2]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [3]:
def check_openai_version():
    """
    Check Azure Open AI version
    """
    installed_version = openai.__version__

    try:
        version_number = float(installed_version[:3])
    except ValueError:
        print("Invalid OpenAI version format")
        return

    print(f"Installed OpenAI version: {installed_version}")

    if version_number < 1.0:
        print("[Warning] You should upgrade OpenAI to have version >= 1.0.0")
        print("To upgrade, run: %pip install openai --upgrade")
    else:
        print(f"[OK] OpenAI version {installed_version} is >= 1.0.0")

In [4]:
check_openai_version()

Installed OpenAI version: 1.30.5
[OK] OpenAI version 1.30.5 is >= 1.0.0


In [5]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    #filter_dict={"tags": ["gpt-4"]},  # comment out to get all
)
# When using a single openai endpoint, you can use the following:
# config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]

In [6]:
WORK_DIR = "workdir"

os.makedirs(WORK_DIR, exist_ok=True)

## 1. Agent

In [7]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

In [8]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
    },
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code.
)

In [9]:
math_problem_to_solve = """
Find $a + b + c$, given that $x+y \\neq -1$ and
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
"""

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=math_problem_to_solve)

user_proxy (to assistant):


Find $a + b + c$, given that $x+y \neq -1$ and
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant (to user_proxy):

This set of equations are non-homogeneous linear equations. We can approach this problem using linear algebra. 

First, we can simplify the three given equations to a standard form 

ax + by + cz = k; where k represents the constant on the right side of the equation.

The equations then become: 

1. ax + by + cz = x+7 
2. bx + cy + az = 2x+6y 
3. cx + ay + bz = 4x+y 

Taking the constants from the right side of the equation to the left side when necessary, we get:

1. (a - 1)x + by + cz = 7 
2. bx + (c - 6)y + az = 2x 
3. cx + ay + (b - 1)z = 4x 

Each equation should be equal to zero for further convenience:

1. (a - 1)x + by + cz - 7 = 0 
2. bx + (c - 6)y + az - 2x = 0 
3. cx + ay + (b - 1)z - 4x =

ChatResult(chat_id=None, chat_history=[{'content': '\nFind $a + b + c$, given that $x+y \\neq -1$ and\n\\begin{align}\n\tax + by + c & = x + 7,\\\n\ta + bx + cy & = 2x + 6y,\\\n\tay + b + cx & = 4x + y.\n\\end{align}.\n', 'role': 'assistant'}, {'content': "This set of equations are non-homogeneous linear equations. We can approach this problem using linear algebra. \n\nFirst, we can simplify the three given equations to a standard form \n\nax + by + cz = k; where k represents the constant on the right side of the equation.\n\nThe equations then become: \n\n1. ax + by + cz = x+7 \n2. bx + cy + az = 2x+6y \n3. cx + ay + bz = 4x+y \n\nTaking the constants from the right side of the equation to the left side when necessary, we get:\n\n1. (a - 1)x + by + cz = 7 \n2. bx + (c - 6)y + az = 2x \n3. cx + ay + (b - 1)z = 4x \n\nEach equation should be equal to zero for further convenience:\n\n1. (a - 1)x + by + cz - 7 = 0 \n2. bx + (c - 6)y + az - 2x = 0 \n3. cx + ay + (b - 1)z - 4x = 0 \n\nLet's

> Now you can interact and say for example to use another library

In [10]:
print(user_proxy.chat_messages[assistant])

[{'content': '\nFind $a + b + c$, given that $x+y \\neq -1$ and\n\\begin{align}\n\tax + by + c & = x + 7,\\\n\ta + bx + cy & = 2x + 6y,\\\n\tay + b + cx & = 4x + y.\n\\end{align}.\n', 'role': 'assistant'}, {'content': "This set of equations are non-homogeneous linear equations. We can approach this problem using linear algebra. \n\nFirst, we can simplify the three given equations to a standard form \n\nax + by + cz = k; where k represents the constant on the right side of the equation.\n\nThe equations then become: \n\n1. ax + by + cz = x+7 \n2. bx + cy + az = 2x+6y \n3. cx + ay + bz = 4x+y \n\nTaking the constants from the right side of the equation to the left side when necessary, we get:\n\n1. (a - 1)x + by + cz = 7 \n2. bx + (c - 6)y + az = 2x \n3. cx + ay + (b - 1)z = 4x \n\nEach equation should be equal to zero for further convenience:\n\n1. (a - 1)x + by + cz - 7 = 0 \n2. bx + (c - 6)y + az - 2x = 0 \n3. cx + ay + (b - 1)z - 4x = 0 \n\nLet's then solve it in Python using sympy l

In [11]:
outputfile = os.path.join(WORK_DIR, "human_interaction.json")

json.dump(user_proxy.chat_messages[assistant], open(outputfile, "w"), indent=6)

In [12]:
with open(outputfile, "r") as f:
    print(f.read())

[
      {
            "content": "\nFind $a + b + c$, given that $x+y \\neq -1$ and\n\\begin{align}\n\tax + by + c & = x + 7,\\\n\ta + bx + cy & = 2x + 6y,\\\n\tay + b + cx & = 4x + y.\n\\end{align}.\n",
            "role": "assistant"
      },
      {
            "content": "This set of equations are non-homogeneous linear equations. We can approach this problem using linear algebra. \n\nFirst, we can simplify the three given equations to a standard form \n\nax + by + cz = k; where k represents the constant on the right side of the equation.\n\nThe equations then become: \n\n1. ax + by + cz = x+7 \n2. bx + cy + az = 2x+6y \n3. cx + ay + bz = 4x+y \n\nTaking the constants from the right side of the equation to the left side when necessary, we get:\n\n1. (a - 1)x + by + cz = 7 \n2. bx + (c - 6)y + az = 2x \n3. cx + ay + (b - 1)z = 4x \n\nEach equation should be equal to zero for further convenience:\n\n1. (a - 1)x + by + cz - 7 = 0 \n2. bx + (c - 6)y + az - 2x = 0 \n3. cx + ay + (b - 1)

In [13]:
!ls $WORK_DIR/*.json -lt

-rwxrwxrwx 1 root root 3247 Jun  4 15:16 workdir/human_interaction.json
